In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

load_dotenv()

def get_coordinates(location_name, GOOGLE_API_KEY):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": location_name,
        "key": GOOGLE_API_KEY
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json().get('results', [])
        if results:
            location = results[0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            raise Exception("No results found for the given location name")
    else:
        raise Exception("Error fetching data from Google Geocoding API")
    
# Example usage
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
location_name = input("Enter a location name: ")
latitude, longitude = get_coordinates(location_name, GOOGLE_API_KEY)
print(f"Coordinates of {location_name}: Latitude = {latitude}, Longitude = {longitude}")


Coordinates of Balaji Symphony: Latitude = 19.0027963, Longitude = 73.1334956


In [2]:
def get_nearby_real_estate_projects(location, radius, GOOGLE_API_KEY):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": location,
        "radius": radius,
        "type": "real_estate_agency",
        "key": GOOGLE_API_KEY
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json().get('results', [])
    else:
        raise Exception("Error fetching data from Google Places API")
    
# Example usage
location = f"{latitude}, {longitude}"
radius = 1000  # in meters
projects = get_nearby_real_estate_projects(location, radius, GOOGLE_API_KEY)

projects_df = pd.DataFrame(projects)
print(projects_df.head(5))


      business_status                                           geometry  \
0         OPERATIONAL  {'location': {'lat': 18.9996697, 'lng': 73.134...   
1         OPERATIONAL  {'location': {'lat': 19.0035034, 'lng': 73.124...   
2         OPERATIONAL  {'location': {'lat': 19.0027963, 'lng': 73.133...   
3         OPERATIONAL  {'location': {'lat': 19.0010466, 'lng': 73.134...   
4  CLOSED_TEMPORARILY  {'location': {'lat': 19.0005746, 'lng': 73.132...   

                                                icon icon_background_color  \
0  https://maps.gstatic.com/mapfiles/place_api/ic...               #7B9EB0   
1  https://maps.gstatic.com/mapfiles/place_api/ic...               #7B9EB0   
2  https://maps.gstatic.com/mapfiles/place_api/ic...               #7B9EB0   
3  https://maps.gstatic.com/mapfiles/place_api/ic...               #7B9EB0   
4  https://maps.gstatic.com/mapfiles/place_api/ic...               #7B9EB0   

                                  icon_mask_base_uri                 name 

In [3]:
## code for the names in projects_df search the names on google for amenitites
print(projects_df['name'])



0                                  KARKERA ENTERPRISES
1                                       Rockstar India
2                                      Balaji Symphony
3                                   Mahajan Properties
4                                       Gatik Realtors
5                          Shree Shubham Estate Agency
6                                Prashil estate agency
7                              Shree Homes Enterprises
8                     RK ROYAL GREEN FACILITY SERVICES
9                                  Shubham Real Estate
10                             Navnath Builders Office
11                                 VISHWAS ENTERPRISES
12    Shree Samarth Krupa Real estate agency,Adai gaon
13                                   Patil Real Estate
14                         I S ENTERPRISES REAL ESTATE
15     Panvel Junction - Property Near Railway Station
16                            MORYA GLOBAL ENTERPRISES
17            K. K. Entarprises Real Estest Consultant
18        

In [4]:
def search_amenities(project_name, serpapi_key):
    url = "https://serpapi.com/search"
    params = {
        "q": project_name + " amenities",
        "api_key": serpapi_key,
        "engine": "google"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json()
        if 'organic_results' in results:
            snippets = [result['snippet'] for result in results['organic_results']]
            return " ".join(snippets) if snippets else "No information found"
        else:
            return "No information found"
    else:
        raise Exception("Error fetching data from SerpAPI")
    
# Example usage
SERPAPI_KEY = os.getenv('SERPAPI_KEY')



# Search for amenities for top 5 projects
for project_name in projects_df['name']:
    amenities = search_amenities(project_name, SERPAPI_KEY)
    print(f"Amenities for {project_name}:\n{amenities}\n")



Amenities for KARKERA ENTERPRISES:
What are some of the property amenities at Karkera Comforts? Some of the more popular amenities offered include free wifi, free parking, and kids activities. In addition to the extensive product line, Karkera Trading Company offers amenities that enhance the overall customer experience. ... enterprises as well as large ... In addition to the extensive product line, Karkera Trading Company offers amenities that enhance the overall customer experience. ... enterprises as well as large ... Utkarsh S. Transforming the hiring ecosystem across industries..! Cloud | Cybersecurity | AI | ML | Data Engineering | Blockchain | Business ... DOMUSS HomeStay offers accommodations in Madikeri, 1.4 miles from Madikeri Fort and 1.8 miles from Raja Seat. This homestay is 3.1 miles from Abbi Falls. Traveller, Food vlogger "FOOD IS THE WAY TO THE HEART" business.seth28@gmail.com. The OSO Programme will develop mixed-use transit-oriented precincts comprising offices, reta

In [5]:
# Dictionary to store project name as key and extracted amenities as value
amenities_data = {}
SERPAPI_KEY = os.getenv('SERPAPI_KEY')



# Search for amenities and store them in the dictionary
for project_name in projects_df['name']:
    amenities = search_amenities(project_name, SERPAPI_KEY)
    amenities_data[project_name] = amenities

# Print the dictionary to see the results
print(amenities_data)


{'KARKERA ENTERPRISES': 'What are some of the property amenities at Karkera Comforts? Some of the more popular amenities offered include free wifi, free parking, and kids activities. In addition to the extensive product line, Karkera Trading Company offers amenities that enhance the overall customer experience. ... enterprises as well as large ... In addition to the extensive product line, Karkera Trading Company offers amenities that enhance the overall customer experience. ... enterprises as well as large ... Utkarsh S. Transforming the hiring ecosystem across industries..! Cloud | Cybersecurity | AI | ML | Data Engineering | Blockchain | Business ... DOMUSS HomeStay offers accommodations in Madikeri, 1.4 miles from Madikeri Fort and 1.8 miles from Raja Seat. This homestay is 3.1 miles from Abbi Falls. Traveller, Food vlogger "FOOD IS THE WAY TO THE HEART" business.seth28@gmail.com. The OSO Programme will develop mixed-use transit-oriented precincts comprising offices, retail, and re

In [6]:
def extract_amenities(text):
    amenities_keywords = [
        "wifi", "free wifi", "internet", "free internet",
        "parking", "car parking", "free parking",
        "gym", "fitness center", "fitness club", "health club",
        "swimming pool", "pool",
        "sauna", "steam room", "jacuzzi",
        "play area", "children play area", "kids activities", "kids zone",
        "garden", "landscape garden", "roof garden", "terrace garden",
        "security", "cctv", "surveillance",
        "club house", "community center", "lounge",
        "power backup", "generator", "24/7 power",
        "luxury bathroom", "modular kitchen", "spacious rooms", "living room"
    ]
    found_amenities = [keyword for keyword in amenities_keywords if keyword in text.lower()]
    return list(set(found_amenities))

# Extract amenities for each project and print them
for project_name, description in amenities_data.items():
    amenities = extract_amenities(description)
    print(f"Amenities for {project_name}:")
    print(", ".join(amenities) if amenities else "No specific amenities found.")
    print()


Amenities for KARKERA ENTERPRISES:
free wifi, free parking, security, parking, kids activities, wifi

Amenities for Rockstar India:
play area, lounge, parking, internet

Amenities for Balaji Symphony:
swimming pool, sauna, jacuzzi, parking, car parking, pool, gym, play area, terrace garden, lounge, landscape garden, garden, steam room

Amenities for Mahajan Properties:
cctv, swimming pool, club house, power backup, 24/7 power, parking, car parking, pool, play area, living room, kids activities, landscape garden, garden

Amenities for Gatik Realtors:
garden

Amenities for Shree Shubham Estate Agency:
modular kitchen, spacious rooms, parking, car parking, living room, luxury bathroom

Amenities for Prashil estate agency:
gym, power backup, parking, car parking

Amenities for Shree Homes Enterprises:
gym, parking, garden, play area

Amenities for RK ROYAL GREEN FACILITY SERVICES:
landscape garden, garden

Amenities for Shubham Real Estate:
gym, lounge, parking, garden

Amenities for Navna

In [9]:
def format_for_llm(df):
    formatted_text = ""
    for index, row in df.iterrows():
        formatted_text += f"{row['Project']} Amenities:\n"
        formatted_text += ", ".join(row['Extracted Amenities']) + "\n\n"
    return formatted_text

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
SERPAPI_KEY = os.getenv('SERPAPI_KEY')

latitude, longitude = get_coordinates(location_name, GOOGLE_API_KEY)
location = f"{latitude}, {longitude}"
radius = 1000  # in meters

projects = get_nearby_real_estate_projects(location, radius, GOOGLE_API_KEY)
projects_df = pd.DataFrame(projects)
amenities_data = {}

for project_name in projects_df['name']:
    amenities = search_amenities(project_name, SERPAPI_KEY)
    amenities_data[project_name] = amenities

df_amenities = pd.DataFrame({
    "Project": list(amenities_data.keys()),
    "Description": list(amenities_data.values())
})

df_amenities['Extracted Amenities'] = df_amenities['Description'].apply(extract_amenities)
formatted_text = format_for_llm(df_amenities)

llm = ChatOllama(model="llama3")

comparison_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that compares amenities of real estate projects."),
        ("human", f"Compare the following amenities:\n\n{formatted_text}")
    ]
)

ranking_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that ranks properties based on their amenities."),
        ("human", f"Rank the following properties based on their amenities considering general preferences and value:\n\n{formatted_text}")
    ]
)

comparison_chain = comparison_prompt | llm
ranking_chain = ranking_prompt | llm

comparison_response = comparison_chain.invoke({"formatted_text": formatted_text})
comparison_result = comparison_response.content.strip()

ranking_response = ranking_chain.invoke({"formatted_text": formatted_text})
ranking_result = ranking_response.content.strip()

print(f"Location: {location_name}")
print(f"Nearby Real Estate Projects: {', '.join(projects_df['name'].head(5))}\n")
print("Comparison Result:\n", comparison_result)
print("\nRanking Result:\n", ranking_result)



Location: Balaji Symphony
Nearby Real Estate Projects: KARKERA ENTERPRISES, Rockstar India, Balaji Symphony, Mahajan Properties, Gatik Realtors

Comparison Result:
 Here's a comparison of the amenities offered by each real estate project:

**Internet/WiFi:**

* Karkera Enterprises: Yes
* Rockstar India: Yes
* I S Enterprises Real Estate: Yes

**Parking/CAR Parking:**

* Karkera Enterprises: Yes
* Rockstar India: Yes
* Balaji Symphony: Yes
* Mahajan Properties: Yes
* Prashil estate agency: Yes
* Shree Homes Enterprises: Yes
* Navnath Builders Office: Yes
* Shubham Real Estate: Yes
* Patil Real Estate: Yes
* Panvel Junction - Property Near Railway Station: Yes

**Gym/Fitness Center:**

* Balaji Symphony: Yes
* Mahajan Properties: Yes
* Prashil estate agency: Yes
* Shree Homes Enterprises: Yes
* K. K. Entarprises Real Estest Consultant: Yes
* BHK homes panvel: Yes
* Patil Real Estate: Yes

**Swimming Pool:**

* Balaji Symphony: Yes
* Mahajan Properties: Yes
* I S Enterprises Real Estate: 